In [ ]:
from google.colab import drive
import sys
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from google.colab import drive
import sys
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# On importe metadata_train
PA_metadata_train=pd.read_parquet("/content/drive/MyDrive/data/metadata/PA_metadata_train.parquet",columns=['surveyId','speciesId'])
PA_metadata_train['speciesId'] = PA_metadata_train['speciesId'].astype(int).astype(str)

# Count the occurrences of each speciesId
species_counts = PA_metadata_train['speciesId'].value_counts()

# Filter the speciesId values that appear more than 300 times
# filtered_species = species_counts[species_counts > 300].index

# Subset the dataset to keep only the rows with the filtered speciesId values
# PA_metadata_train = PA_metadata_train[PA_metadata_train['speciesId'].isin(filtered_species)]

# Concatenante in list all speciesId for each surveyId
PA_metadata_train = PA_metadata_train.groupby('surveyId')['speciesId'].apply(list).reset_index()

# On importe metadata_train
PA_metadata_test=pd.read_parquet("/content/drive/MyDrive/data/metadata/PA_metadata_test.parquet",columns=['surveyId'])

#train_labels_list = [[label] for label in train_labels]
#val_labels_list = [[label] for label in val_labels]

In [ ]:
def get_image_paths(survey_id, rgb_dir, nir_dir):
    # Convert the survey_id to a string
    survey_id_str = str(survey_id)

    # Extract the last 4 characters and split them into two subdirectories
    sub_dir1 = survey_id_str[-4:-2]
    sub_dir2 = survey_id_str[-2:]

    # Create the image name using the last 6 characters of the survey_id
    image_name = f"{survey_id_str}.jpeg"

    # Construct the RGB image path by joining the rgb_dir, sub_dir2, sub_dir1, and image_name
    rgb_image_path = os.path.join(rgb_dir, sub_dir2, sub_dir1, image_name)

    # Construct the NIR image path by joining the nir_dir, sub_dir2, sub_dir1, and image_name
    nir_image_path = os.path.join(nir_dir, sub_dir2, sub_dir1, image_name)

    # Return the RGB and NIR image paths as a tuple
    return rgb_image_path, nir_image_path

def create_dataset_train(metadata_df, rgb_dir, nir_dir):
    # Initialize empty lists to store RGB image paths, NIR image paths, and labels
    rgb_image_paths = []
    nir_image_paths = []

    labels = []

    # Iterate over each row in the metadata DataFrame
    for _, row in metadata_df.iterrows():
        # Get the survey_id from the current row
        survey_id = row['surveyId']

        # Call the get_image_paths function to get the RGB and NIR image paths for the current survey_id
        rgb_path, nir_path = get_image_paths(survey_id, rgb_dir, nir_dir)

        # Append the RGB image path to the rgb_image_paths list
        rgb_image_paths.append(rgb_path)

        # Append the NIR image path to the nir_image_paths list
        nir_image_paths.append(nir_path)

        # Append the speciesId from the current row to the labels list
        labels.append(row['speciesId'])

    # Return the lists of RGB image paths, NIR image paths, and labels as a tuple
    return rgb_image_paths, nir_image_paths, labels

def create_dataset_test(metadata_df, rgb_dir, nir_dir):
  # Initialize empty lists to store RGB image paths, NIR image paths, and labels
  rgb_image_paths = []
  nir_image_paths = []

  # Iterate over each row in the metadata DataFrame
  for _, row in metadata_df.iterrows():
      # Get the survey_id from the current row
      survey_id = row['surveyId']

      # Call the get_image_paths function to get the RGB and NIR image paths for the current survey_id
      rgb_path, nir_path = get_image_paths(survey_id, rgb_dir, nir_dir)

      # Append the RGB image path to the rgb_image_paths list
      rgb_image_paths.append(rgb_path)

      # Append the NIR image path to the nir_image_paths list
      nir_image_paths.append(nir_path)

  # Return the lists of RGB image paths, NIR image paths, and labels as a tuple
  return rgb_image_paths, nir_image_paths

In [ ]:
# Create the PA train dataset
rgb_train_paths, nir_train_paths, train_labels = create_dataset_train(
    PA_metadata_train,
    rgb_dir='/content/drive/MyDrive/data/SatellitePatches/pa_train_patches_rgb',
    nir_dir='/content/drive/MyDrive/data/SatellitePatches/pa_train_patches_nir'
)

rgb_test_paths, nir_test_paths = create_dataset_test(
    PA_metadata_test,
    rgb_dir='/content/drive/MyDrive/data/SatellitePatches/pa_test_patches_rgb',
    nir_dir='/content/drive/MyDrive/data/SatellitePatches/pa_test_patches_nir'
)

# Split the paths and labels into train and validation sets
#rgb_train_paths, rgb_val_paths, nir_train_paths, nir_val_paths, train_labels, val_labels = train_test_split(
#    rgb_train_paths, nir_train_paths, train_labels, test_size=0.2, random_state=42
#)


In [ ]:
#train_labels_list = [[label] for label in train_labels]
#val_labels_list = [[label] for label in val_labels]

In [ ]:
# Convert labels to multi-label binary format
mlb = MultiLabelBinarizer()
train_labels_bin = mlb.fit_transform(train_labels)
#val_labels_bin = mlb.transform(val_labels)

In [ ]:
def load_and_preprocess_image_train(rgb_path, nir_path, label):
    rgb_image = tf.io.read_file(rgb_path)
    rgb_image = tf.image.decode_jpeg(rgb_image, channels=3)
    rgb_image = tf.image.resize(rgb_image, (128, 128))  # Resize to match the input shape
    rgb_image = tf.cast(rgb_image, tf.float32) / 255.0  # Normalize pixel values

    nir_image = tf.io.read_file(nir_path)
    nir_image = tf.image.decode_jpeg(nir_image, channels=1)  # Load as grayscale image with a single channel
    nir_image = tf.image.resize(nir_image, (128, 128))  # Resize to match the input shape
    nir_image = tf.cast(nir_image, tf.float32) / 255.0  # Normalize pixel values

    return (rgb_image, nir_image), label  # Return a single tuple containing the RGB image, NIR image, and label

def load_and_preprocess_image_test(rgb_path, nir_path):
  rgb_image = tf.io.read_file(rgb_path)
  rgb_image = tf.image.decode_jpeg(rgb_image, channels=3)
  rgb_image = tf.image.resize(rgb_image, (128, 128))  # Resize to match the input shape
  rgb_image = tf.cast(rgb_image, tf.float32) / 255.0  # Normalize pixel values

  nir_image = tf.io.read_file(nir_path)
  nir_image = tf.image.decode_jpeg(nir_image, channels=1)  # Load as grayscale image with a single channel
  nir_image = tf.image.resize(nir_image, (128, 128))  # Resize to match the input shape
  nir_image = tf.cast(nir_image, tf.float32) / 255.0  # Normalize pixel values

  return (rgb_image, nir_image)  # Return a single tuple containing the RGB image and NIR image


PA_train_dataset = tf.data.Dataset.from_tensor_slices((
    tf.constant(rgb_train_paths, dtype=tf.string),
    tf.constant(nir_train_paths, dtype=tf.string),
    tf.constant(train_labels_bin, dtype=tf.int64)
))

PA_test_dataset = tf.data.Dataset.from_tensor_slices((
    tf.constant(rgb_test_paths, dtype=tf.string),
    tf.constant(nir_test_paths, dtype=tf.string)
))

PA_train_dataset = PA_train_dataset.map(load_and_preprocess_image_train).cache()
PA_test_dataset = PA_test_dataset.map(load_and_preprocess_image_test).cache()

batch_size = 256
PA_train_dataset = PA_train_dataset.shuffle(buffer_size=2000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = PA_test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
#PA_val_dataset = PA_val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Assume you have already created the list of RGB image paths named 'rgb_image_paths'

# Count the number of existing and non-existing paths
#existing_paths = 0
#non_existing_paths = 0

#for path in rgb_train_paths:
#    if os.path.exists(path):
#        existing_paths += 1
#    else:
#        non_existing_paths += 1

#print(f"Number of existing paths: {existing_paths}")
#print(f"Number of non-existing paths: {non_existing_paths}")

Number of existing paths: 1483637
Number of non-existing paths: 0


In [ ]:
def visualize_dataset(dataset, num_samples):
    plt.figure(figsize=(10, 10))
    for i, (rgb_image_path, nir_image_path, label) in enumerate(dataset.shuffle(1000).take(num_samples)):
        # Read RGB image from path
        rgb_image = tf.io.read_file(rgb_image_path.numpy().decode('utf-8'))
        rgb_image = tf.image.decode_jpeg(rgb_image, channels=3)

        # Read NIR image from path
        nir_image = tf.io.read_file(nir_image_path.numpy().decode('utf-8'))
        nir_image = tf.image.decode_jpeg(nir_image, channels=1)
        nir_image = tf.image.grayscale_to_rgb(nir_image)

        # Display RGB image
        plt.subplot(2, num_samples, i+1)
        plt.imshow(rgb_image.numpy().astype("uint8"))
        plt.title(f"RGB - Label: {label.numpy()}")
        plt.axis("off")

        # Display NIR image
        plt.subplot(2, num_samples, i+num_samples+1)
        plt.imshow(nir_image.numpy().astype("uint8"))
        plt.title(f"NIR - Label: {label.numpy()}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# Assume you have already created the train_dataset using the previous code

# Preprocess the dataset for visualization
#vis_dataset = PA_train_datasets.map(lambda rgb_image_path, nir_image_path, label: (rgb_image_path, nir_image_path, label))

# Visualize a few random samples from the dataset
#num_samples_to_visualize = 5
#visualize_dataset(vis_dataset, num_samples_to_visualize)

# Transfer Learning with Resnet-50 on RGB and NIR model

## Définition des modèles individuelles

In [ ]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from tensorflow.keras import Input,Sequential,Model,models
from tensorflow.keras.layers import Lambda, Flatten, BatchNormalization, Dense, Dropout, Concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback

In [ ]:
# Définition de la taille cible des images en entrée du modèle
count_species=train_labels_bin.shape[1]

# Définition des entrées du modèle
rgb_input = Input(shape=(128,128, 3), name='rgb_input') # RGB donc 3 canaux
nir_input = Input(shape=(128,128, 1), name='nir_input') # NIR, nuance de gris donc un canal

# Conversion de l'image NIR en image RGB
# Resnet a été entrainé sur du RGB avec les weights provenant de imagenet
nir_rgb = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(nir_input)

# On définit les deux modèles ResNet, il faut en créer deux pour éviter
# la duplication des noms dans les layers
rgb_resnet_model = ResNet50(weights=None, include_top=False, input_tensor=rgb_input)
nir_resnet_model = ResNet50(weights=None, include_top=False, input_tensor=nir_rgb)

# Freeze the layers of the ResNet50 base model for RGB model
for model in [rgb_resnet_model,nir_resnet_model]:
  for layer in model.layers[:143]:
      layer.trainable = False

# Define the RGB model
rgb_model = Sequential()
rgb_model.add(rgb_resnet_model)
rgb_model.add(Flatten())
rgb_model.add(BatchNormalization())
rgb_model.add(Dense(256, activation='relu'))
rgb_model.add(Dropout(0.5))
rgb_model.add(BatchNormalization())
rgb_model.add(Dense(128, activation='relu'))
rgb_model.add(Dropout(0.5))
rgb_model.add(BatchNormalization())
rgb_model.add(Dense(64, activation='relu'))
rgb_model.add(Dropout(0.5))
rgb_model.add(BatchNormalization())

# Define the NIR model
nir_model = Sequential()
nir_model.add(nir_resnet_model)
nir_model.add(Flatten())
nir_model.add(BatchNormalization())
nir_model.add(Dense(256, activation='relu'))
nir_model.add(Dropout(0.5))
nir_model.add(BatchNormalization())
nir_model.add(Dense(128, activation='relu'))
nir_model.add(Dropout(0.5))
nir_model.add(BatchNormalization())
nir_model.add(Dense(64, activation='relu'))
nir_model.add(Dropout(0.5))
nir_model.add(BatchNormalization())

# Concatenate the two models
concatenated = Concatenate()([rgb_model(rgb_input), nir_model(nir_input)])

# Define the output layer
output = Dense(count_species, activation='sigmoid')(concatenated)

# Define the model
patch_model = Model(inputs=[rgb_input, nir_input], outputs=output)

## Définition callbacks et autres paramètres

In [ ]:
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

#Définition d'un callback de sauvegarde du modèle
# Ce callback permet de sauvegarder le modèle à chaque époque si la précision sur le jeu de validation a augmenté par rapport à l'époque précédente

# Création du callback ModelCheckpoint
check_point = callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/mémoire/geolife_model_1505",
                                        monitor="loss",
                                        mode="min",
                                        save_best_only=True,
                                        overwrite=True)

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='loss',  # monitor training loss
    min_delta=0,  # minimum change in loss to qualify as an improvement
    patience=3,  # number of epochs with no improvement after which training will be stopped
    verbose=1  # verbosity mode
)


# Explication des paramètres utilisés :
# - filepath : chemin d'accès au fichier où le modèle sera sauvegardé. Dans ce cas, le fichier s'appellera "cifar10.h5".
# - monitor : métrique à surveiller pour décider si le modèle doit être sauvegardé ou non. Dans ce cas, on surveille la précision sur le jeu de validation ("val_acc").
# - mode : mode de surveillance de la métrique. Dans ce cas, on souhaite maximiser la précision sur le jeu de validation, donc on utilise "max".
# - save_best_only : booléen indiquant si seul le meilleur modèle doit être sauvegardé ou non. Dans ce cas, on utilise "True" pour ne sauvegarder que le meilleur modèle.

In [ ]:
from keras import metrics

# Appel de la méthode compile du modèle
patch_model.compile(
    optimizer='adam',  # Utilisation de l'optimiseur Adam
    loss='binary_crossentropy',  # Utilisation de la perte 'binary_crossentropy' pour un problème de classification binaire
    metrics=['binary_accuracy']  # Utilisation des métriques binary_accuracy et F1-score
)

In [ ]:
epochs = 15
history = patch_model.fit(
    PA_train_dataset,  # Provide both inputs and labels to the model
    epochs=epochs,
    callbacks=[check_point, early_stopping]
  )

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.summary()
model.save("geolife.h5")

In [ ]:
# Initialize an empty DataFrame
predictions_df = pd.DataFrame(columns=['surveyId', 'top_25_predictions'])

# Make predictions on the test dataset
for i, (rgb_image, nir_image, survey_id) in enumerate(test_dataset):
    prediction = patch_model.predict([rgb_image, nir_image])

    # Sort the predictions by probability in descending order and select the top 25
    top_25_indices = np.argsort(prediction)[-25:]
    top_25_predictions = prediction[top_25_indices]
    top_25_labels = mlb.classes_[top_25_indices]

    # Convert the top 25 predictions to a space-separated string
    top_25_string = ' '.join(map(str, top_25_labels))

    # Add the surveyId and top_25_predictions to the DataFrame
    new_row = pd.DataFrame({'surveyId': [survey_id], 'top_25_predictions': [top_25_string]})
    predictions_df = pd.concat([predictions_df, new_row], ignore_index=True)